## We want to build a model of demand and supply of publicly provided services in a city

In [1]:
import geopy
import numpy as np
import pandas as pd
from sklearn import gaussian_process
from matplotlib import pyplot as plt 
from enum import Enum


In [2]:
gaussKern = gaussian_process.kernels.RBF

In [64]:
## Enum classes
class AgeGroup(Enum):
    Newborn= (0, 3)
    Child= (4,15)
    Young= (16,25)
    Junior= (26,35)
    Senior= (36, 50)
    Over50= (50, 64)
    Over65= (66, 80)
    Over80= (81, 200)
    def __init__(self, startAge, endAge):
        self.start = startAge
        self.end = endAge
    
    @staticmethod
    def all():
        return([g for g in AgeGroup])
    @property
    def range(self): return self.end - self.start
    
class SummaryNorm(Enum):
    l1= lambda x: sum(abs(x))
    l2= lambda x: (sum(x**2))**0.5
    lInf= lambda x: max(x)
    

class ServiceType(Enum):
    School = (1, SummaryNorm.l2)
    Football = (2, SummaryNorm.l2)
    SocialSupport = (3, SummaryNorm.l2)
    PoliceStation = (4, SummaryNorm.l2)
    #etc
    def __init__(self, _, aggrNormInput=SummaryNorm.l2):
        self.aggrNorm = aggrNormInput
    
    def aggregate_units(self, x, axis=1):
        # assumes position are stacked in rows
        return np.apply_along_axis(self.aggrNorm, axis, x)

In [66]:
## ServiceUnit class
class ServiceUnit:
    def __init__(self, service, name='', position=(45.4641, 9.1919), users=AgeGroup.all(), times=[], scale=1):
        assert isinstance(position, tuple) & (len(position) == 2), 'Position must be a pair tuple' #will be replaced by nicer class
        assert isinstance(service, ServiceType), 'Service must belong to the Enum'
        self.name = name
        self.service = service
        self.site = position  # a Service can have many sites, and a site is not uniquely assigned to a service
        self.times = times
        
        # how the service availablity area varies for different age groups
        kPropagationTest = 3
        self.propagation ={g: (0.04 + .001*np.round(np.random.normal(),2))*kPropagationTest*scale for g in users} 
        self.kernel = {g: gaussKern(length_scale=l) for g, l in self.propagation.items()}
        
    def evaluate(self, targetPositions, ageGroup):
        assert targetPositions.shape[1] == 2, '2D targets expected'
        reshapedPos = np.array(self.site).reshape(-1,2)
        # evaluate kernel to get level service score. If age group is not served, return 0 as default
        if self.kernel.__contains__(ageGroup):
            score = self.kernel[ageGroup](targetPositions, reshapedPos)
        else:
            score = 0
        return np.squeeze(score)
    
    @property
    def users(self): return list(self.propagation.keys())
    
def evaluate_services_at(positions, unitsList, serviceTypes= [t for t in ServiceType]):
    # set all age groups as output default
    outputAgeGroups = AgeGroup.all()
    
    outScores = np.zeros((positions.shape[0], len(outputAgeGroups), len(serviceTypes)), 
                         dtype=[('position', tuple), ('age', AgeGroup), ('service', ServiceType)])
    # for a given position, output is a (ageGroup VS serviceType) table. Let's initialise it
    #outScores = pd.DataFrame(np.zeros([len(outputAgeGroups), len(serviceTypes)]),
    #                         index=outputAgeGroups, columns=serviceTypes)
    for thisServType in serviceTypes:
        for thisAgeGroup in outputAgeGroups:
            serviceUnits = [u for u in unitsList if u.service == thisServType]
            if not serviceUnits:
                continue
            #print(np.squeeze(serviceUnits[0].evaluate(positions, thisAgeGroup)))
            unitValues = np.stack(list(map(lambda x: x.evaluate(positions, thisAgeGroup), serviceUnits)),
                                 axis=-1)
            outScores.loc[thisAgeGroup, thisServType] = thisServType.aggregate_units(unitValues)
    return outScores
        

In [67]:
test = [ServiceUnit(ServiceType.PoliceStation, 'Duomo'), ServiceUnit(ServiceType.PoliceStation, 'Ripamonti')]
x = np.random.uniform(low=[45.40,9.1],high=[45.50, 9.3], size=(50,2))
evaluate_services_at(x, test)


AttributeError: 'numpy.ndarray' object has no attribute 'loc'

In [ ]:
values = np.zeros([len(AgeGroup.all()), len([t for t in ServiceType])])
zz=pd.DataFrame(values, index= AgeGroup.all(), columns = [t for t in ServiceType])

In [ ]:
zz.loc[AgeGroup.Newborn, ServiceType.PoliceStation]

In [ ]:
np.array((3,4))

In [ ]:
 [t for t in ServiceType]

In [ ]:
### Supply modeling
